In [1]:
import docx
from glob import glob
import re
import os
import unicodedata
from firecrawl import FirecrawlApp

from diacritics import *

In [206]:
def read_docx(path: str) -> str:
    doc = docx.Document(path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def generate_tokens(text: str) -> list[str]:
    tokens = re.split(r'([^\w\u0300-\u036f\s]|\s+)', text)
    tokens = [t for t in tokens if t]
    return tokens

def join_tokens(tokens: list[str]) -> str:
    return ''.join(tokens)

def remove_diacritics(word: str) -> str:
    normalized = unicodedata.normalize('NFKD', word)
    return ''.join(c for c in normalized if not unicodedata.combining(c))

def add_diacritics(word: str, mappings: dict[str, str]) -> str:
    word_with_diacritics = mappings.get(word.lower(), word)

    if word.isupper():
        return word_with_diacritics.upper()
    elif word.islower():
        return word_with_diacritics.lower()
    elif word[0].isupper():
        return word_with_diacritics[0].upper() + word_with_diacritics[1:]
    else:
        raise ValueError(f"Cannot add diacritics to word: {word} with mapping: {word_with_diacritics}")
    
def make_mappings(tokens: list[str]) -> dict[str, str]:
    mappings = {}
    for token in tokens:
        token_without_diacritics = remove_diacritics(token)
        if (token != token_without_diacritics) and not token.isspace():
            mappings[token_without_diacritics.lower()] = token.lower()
    return mappings

def reconstruct_tokens(tokens: list[str], mappings: dict[str, str]) -> list[str]:
    reconstructed_tokens = []
    for token in tokens:
        if token not in mappings:
            reconstructed_tokens.append(token)
        else:
            reconstructed_tokens.append(add_diacritics(token, mappings))
    return reconstructed_tokens

def verify(text: str, reconstructed_text: str, tokens: list[str], reconstructed_tokens: list[str]) -> bool:
    if text != reconstructed_text:

        #do length check on tokens
        if len(tokens) != len(reconstructed_tokens):
            print(f"length mismatch in {path}")
            print(f"tokens: {len(tokens)}, reconstructed_tokens: {len(reconstructed_tokens)}")

        #do length check
        if len(text) != len(reconstructed_text):
            print(f"length mismatch in {path}")
            print(f"text: {len(text)}, reconstructed_text: {len(reconstructed_text)}")

        for orig, recon in zip(tokens, reconstructed_tokens):
            if orig != recon:
                print(f"original: {orig}, reconstructed: {recon}")
        return False
    else:
        print(f"verification successful for {path}")
        return True

def test_txt_file(path: str) -> bool:
    text = open(path, "r").read()
    tokens = generate_tokens(text)
    mappings = make_mappings(tokens)
    reconstructed_tokens = reconstruct_tokens(tokens, mappings)
    reconstructed_text = join_tokens(reconstructed_tokens)
    return verify(text, reconstructed_text, tokens, reconstructed_tokens)

In [3]:
paths = glob("data/txts/*.txt")
for path in paths:
    test_txt_file(path)


original: Shri, reconstructed: Shrī
text: 2918, reconstructed_text: 2919
original: Shri, reconstructed: Shrī
original: Durva, reconstructed: Durvā
text: 15065, reconstructed_text: 15072
original: Paksha, reconstructed: Paks̲h̲a
original: Paksha, reconstructed: Paks̲h̲a
original: NAKSHATRA, reconstructed: NAKS̲H̲ATRA
original: CHITRA, reconstructed: CHITRĀ
original: JYESHTHA, reconstructed: JYESHTHĀ
original: Bali, reconstructed: Balī
original: Bali, reconstructed: Balī
original: Bali, reconstructed: Balī
original: samadhi, reconstructed: samād͟hi
original: Chaturdashi, reconstructed: Chaturdashī
original: Boita, reconstructed: Boitā
original: Boita, reconstructed: Boitā
original: Bali, reconstructed: Balī
text: 6989, reconstructed_text: 6990
original: Math, reconstructed: Mat͟h
original: Pune, reconstructed: Puṇe
text: 11755, reconstructed_text: 11820
original: Shri, reconstructed: Shrī
original: of, reconstructed: ̄of
original: of, reconstructed: ̄of
original: of, reconstructed: ̄of

In [188]:
urls = ["https://chitrapurmath.net/site/about-introduction",
"https://chitrapurmath.net/site/about-introduction-math",
"https://chitrapurmath.net/site/about-introduction-journey",
"https://chitrapurmath.net/site/about-parampara",
"https://chitrapurmath.net/gauravam?id=0",
"https://chitrapurmath.net/gauravam?id=1",
"https://chitrapurmath.net/gauravam?id=2",
"https://chitrapurmath.net/gauravam?id=3",
"https://chitrapurmath.net/gauravam?id=4",
"https://chitrapurmath.net/gauravam?id=5",
"https://chitrapurmath.net/gauravam?id=6",
"https://chitrapurmath.net/gauravam?id=7",
"https://chitrapurmath.net/gauravam?id=8",
"https://chitrapurmath.net/gauravam?id=9",
"https://chitrapurmath.net/gauravam?id=10",
"https://chitrapurmath.net/gauravam?id=11",
"https://chitrapurmath.net/gauravam?id=12",
"https://chitrapurmath.net/gauravam?id=13",
"https://chitrapurmath.net/gauravam?id=14",
"https://chitrapurmath.net/gauravam?id=15",
"https://chitrapurmath.net/news/default/story?id=66",
"https://chitrapurmath.net/news/default/story?id=168",
"https://chitrapurmath.net/news/default/story?id=48",
"https://chitrapurmath.net/news/default/story?id=213",
"https://chitrapurmath.net/news/default/story?id=229",
"https://chitrapurmath.net/news/default/story?id=253",
"https://chitrapurmath.net/news/default/story?id=90",
"https://chitrapurmath.net/news/default/story?id=13",
"https://chitrapurmath.net/news/default/story?id=172",
"https://chitrapurmath.net/news/default/story?id=164",
"https://chitrapurmath.net/news/default/story?id=271",
"https://chitrapurmath.net/news/default/story?id=1",
"https://chitrapurmath.net/news/default/story?id=88",
"https://chitrapurmath.net/news/default/story?id=74",
"https://chitrapurmath.net/news/default/story?id=68",
"https://chitrapurmath.net/news/default/story?id=277",
"https://chitrapurmath.net/news/default/story?id=15",
"https://chitrapurmath.net/news/default/story?id=161",
"https://chitrapurmath.net/news/default/story?id=107",
"https://chitrapurmath.net/news/default/story?id=207",
"https://chitrapurmath.net/news/default/story?id=266",
"https://chitrapurmath.net/news/default/story?id=5",
"https://chitrapurmath.net/news/default/story?id=7?",
"https://chitrapurmath.net/news/default/story?id=69",
"https://chitrapurmath.net/news/default/story?id=226",
"https://chitrapurmath.net/news/default/story?id=255",
"https://chitrapurmath.net/news/default/story?id=250",
"https://chitrapurmath.net/news/default/story?id=160",
"https://chitrapurmath.net/news/default/story?id=58",
"https://chitrapurmath.net/news/default/story?id=202",
"https://chitrapurmath.net/news/default/story?id=205",
"https://chitrapurmath.net/news/default/story?id=45",
"https://chitrapurmath.net/news/default/story?id=126",
"https://chitrapurmath.net/news/default/story?id=151",
"https://chitrapurmath.net/news/default/story?id=264",
"https://chitrapurmath.net/news/default/story?id=247",
"https://chitrapurmath.net/news/default/story?id=22",
"https://chitrapurmath.net/news/default/story?id=189",
"https://chitrapurmath.net/news/default/story?id=18",
"https://chitrapurmath.net/news/default/story?id=78",
"https://chitrapurmath.net/news/default/story?id=92",
"https://chitrapurmath.net/news/default/story?id=42",
"https://chitrapurmath.net/news/default/story?id=221",
"https://chitrapurmath.net/news/default/story?id=243",
"https://chitrapurmath.net/news/default/story?id=79",
"https://chitrapurmath.net/news/default/story?id=169",
"https://chitrapurmath.net/news/default/story?id=265",
"https://chitrapurmath.net/news/default/story?id=153",
"https://chitrapurmath.net/news/default/story?id=39?",
"https://chitrapurmath.net/news/default/story?id=8?",
"https://chitrapurmath.net/news/default/story?id=281",
"https://chitrapurmath.net/news/default/story?id=216",
"https://chitrapurmath.net/news/default/story?id=278",
"https://chitrapurmath.net/news/default/story?id=245",
"https://chitrapurmath.net/news/default/story?id=23",
"https://chitrapurmath.net/news/default/story?id=251",
"https://chitrapurmath.net/news/default/story?id=103",
"https://chitrapurmath.net/news/default/story?id=2",
"https://chitrapurmath.net/news/default/story?id=81",
"https://chitrapurmath.net/news/default/story?id=47",
"https://chitrapurmath.net/news/default/story?id=102",
"https://chitrapurmath.net/news/default/story?id=156",
"https://chitrapurmath.net/news/default/story?id=263",
"https://chitrapurmath.net/news/default/story?id=19",
"https://chitrapurmath.net/news/default/story?id=118",
"https://chitrapurmath.net/news/default/story?id=159",
"https://chitrapurmath.net/news/default/story?id=75",
"https://chitrapurmath.net/news/default/story?id=20",
"https://chitrapurmath.net/news/default/story?id=56",
"https://chitrapurmath.net/news/default/story?id=136",
"https://chitrapurmath.net/news/default/story?id=283",
"https://chitrapurmath.net/news/default/story?id=30",
"https://chitrapurmath.net/news/default/story?id=46",
"https://chitrapurmath.net/news/default/story?id=187",
"https://chitrapurmath.net/news/default/story?id=201",
"https://chitrapurmath.net/news/default/story?id=246",
"https://chitrapurmath.net/news/default/story?id=76",
"https://chitrapurmath.net/news/default/story?id=52",
"https://chitrapurmath.net/news/default/story?id=222",
"https://chitrapurmath.net/news/default/story?id=70",
"https://chitrapurmath.net/news/default/story?id=10",
"https://chitrapurmath.net/news/default/story?id=252",
"https://chitrapurmath.net/news/default/story?id=241",
"https://chitrapurmath.net/news/default/story?id=269",
"https://chitrapurmath.net/news/default/story?id=87",
"https://chitrapurmath.net/news/default/story?id=71",
"https://chitrapurmath.net/news/default/story?id=4",
"https://chitrapurmath.net/news/default/story?id=44",
"https://chitrapurmath.net/news/default/story?id=208",
"https://chitrapurmath.net/news/default/story?id=32?",
"https://chitrapurmath.net/news/default/story?id=57",
"https://chitrapurmath.net/news/default/story?id=210",
"https://chitrapurmath.net/news/default/story?id=3",
"https://chitrapurmath.net/news/default/story?id=223",
"https://chitrapurmath.net/news/default/story?id=163",
"https://chitrapurmath.net/news/default/story?id=154",
"https://chitrapurmath.net/news/default/story?id=254",
"https://chitrapurmath.net/news/default/story?id=273",
"https://chitrapurmath.net/news/default/story?id=270",
"https://chitrapurmath.net/news/default/story?id=174",
"https://chitrapurmath.net/news/default/story?id=197",
"https://chitrapurmath.net/news/default/story?id=59",
"https://chitrapurmath.net/news/default/story?id=166",
"https://chitrapurmath.net/news/default/story?id=17"]

In [218]:
#save all urls to txt file
with open("urls.txt", "w") as f:
    for url in urls:
        f.write(url + "\n")

In [ ]:

FIRECRAWL_API_KEY = "fc-167746d1f6664c38b2c7824111f3f513"

app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)

for i, url in enumerate(urls):
    print(f"scraping {url}")
    if os.path.exists(f"scrape/{i}.txt"):
        print(f"skipping {url} because it already exists")
        continue
    scrape = app.scrape_url(
        url, 
        params={'formats': ['markdown']}
    )
    save_path = f"scrape/{i}.txt"
    with open(save_path, "w") as f:
        f.write(scrape['markdown'])

In [ ]:
paths = glob("scrape/*.txt")
for path in paths:
    test_txt_file(path)

In [ ]:
#get all mappings
paths = glob("scrape/*.txt")
all_mappings = []
for path in paths:
    print(path)
    text = open(path, "r").read()
    tokens = generate_tokens(text)
    mappings = make_mappings(tokens)
    if "zia" in mappings:
        print(mappings["zia"])
    all_mappings.append(mappings)

all_mappings = {k: v for d in all_mappings for k, v in d.items()}

# print(all_mappings)
# with open("all_mappings.txt", "w") as f:
#     for k, v in all_mappings.items():
#         f.write(f"{k},{v}\n")

In [ ]:
[k for k,v in all_mappings.items() if k.endswith("shram")]

In [ ]:
#load from all_mappings.txt
all_mappings = {}
with open("all_mappings.txt", "r") as f:
    seen_keys = set()
    for line in f:
        k, v = line.strip().split(",")
        # Normalize to ASCII - keep only chars from space (0x20) to 'z' (0x7A)
        normalized_k = ''.join(c for c in k if 0x20 <= ord(c) <= 0x7A)
        if normalized_k != k:
            continue
        seen_keys.add(normalized_k)
        all_mappings[normalized_k] = v

all_mappings_new = {}
for k,v in all_mappings.items():
    all_mappings_new[unicodedata.normalize('NFKD', k).encode('ascii', 'ignore').decode('ascii')] = v

print(len(all_mappings))
print(len(all_mappings_new))


In [232]:
#save all mappings to txt file
with open("all_mappings_new.txt", "w") as f:
    for k, v in all_mappings_new.items():
        f.write(f"{k},{v}\n")

In [4]:
from app import main

In [5]:
text = """
Forty days in Divine Sannidhya of the Guru - Our Uttar Bharat Yatra
A Report by Vinay Kalyanpur

Preparations for the Uttar Bharat Yatra started months in advance! The very thought was a bit overwhelming for me – 6000 kms to cover by road in 40 days with seven unknown destinations to visit! But I made up my mind to join only because it was initiated by H.H. Swamiji and we would be travelling along with Him. In retrospect now, I am so happy I decided to go – my first yatra with our Math team and I enjoyed every moment of it!
18 of us yatri-s got together at Karla where our travel companions from Shirali had already arrived with Animay and Manju in a comfortable 25 seater bus. These two young drivers would be in charge of driving us around throughout this long but well planned yatra. 12th February 2023, the historic day arrived none too soon and we started our yatra following H.H. Swamiji’s convoy at 8.58 a.m. from Karla.
Our first destination – Shankaraloka, a very serene and beautiful Αshram managed by Samvit Sadhanayana, with a temple dedicated to Bhagavan Shankar along with Devi Parvati, a Navagraha sanctuary where the Navagraha vigraha-s reside among their respective vanaspati. 
H.H. Swamiji and His entourage arrived at Shankaraloka by 11.30 p.m. on the 12th. While H.H. Swamiji was accommodated at Shankaraloka Αshram itself, the entourage was put up at the comfortable Trimandir, a short distance away.
Over the next three days followed informal interactions, Swadhyaya, Αshirvachana-s, soulful Bhajana-s, etc. in the divine sannidhya of our beloved Guru. These precious moments continued to pervade the entire yatra at every single destination, in addition to pre-planned programmes by the local sadhaka-s.
For me personally, there were many ‘Firsts’ in this yatra - my first time in a yatra with H.H. Swamiji and His team and my first-time attendance at Swadhyaya, too! To my utter delight, Pujya Swamiji took up the lovely Gauri-Dashakam stotram for Swadhyaya on two days – I learnt that aradhana of Shakti is important for a sadhaka because without Her Anugraha, one cannot hope to reach Shiva, his ultimate goal.
On both these days, post Swadhyaya, an engaging Sanskrit Sambhashanam session was conducted by Shilpa Mudur pachi. She narrated, nay, acted out a mythological story in simple Sanskrit followed by a question-answer round. In spite of not knowing Sanskrit at all, I was amazed that all of us could understand it very well indeed! Kudos to Shilpa pachi’s immense prowess!
The evenings were invariably precious with the deep messages delivered in simple words by H.H. Swamiji in His Upadesha, be it an informal Satsanga or the formal Αshirvachana.
Our erudite Dharmapracharaka – Dr. Chaitanya Gulvady mam presented a Kirtana on Sant Namdev. It was a pleasant surprise to most of us who were unaware of this wonderful aspect of his humble personality. He came dressed to the part and left us spellbound with his skillful and soulful narration. Another highlight at Shankaraloka was the presentation on the Kailash Manasarovar Yatra (2002) undertaken by H.H. Swamiji along with 96 yatri-s that Chaitanya Gulvady mam showed through a PPT in his very engaging, inimitable style full of devotion.
I would recommend anyone and everyone even remotely interested in Kailash to see this presentation whenever possible as it gives a first-hand perspective of the crises, challenges, progress and finally success of the entire journey made in a span of 20 days which should make us Chitrapur Saraswat-s very proud. Personally, I found this talk taking a similar trajectory to one’s everyday life which is colourful, going through its own unpredictable ups and downs, but with our firm faith in our Guru and Lord Bhavanishankar, we too can overcome any difficulty and the end result shall be memorable!
On 15th February, our last day at Shankaraloka, concluded with the Dharma Sabha –a fulfilling occasion for the local sadhaka-s to offer Paduka Pujana and be blessed with an illuminating Αshirvachana by both Swami Narayan Giriji Maharaj and H.H. Swamiji. It was indeed a moment of joy and pride for us to hear Swami Narayan Giriji recounting his days at Sant Sarovar witnessing the dedicated tapas and sadhana that Pujya Sadyojat Shankarashram Swamiji underwent joyfully to reach this unparalleled stature of an ideal Sanyasi and move on to become an ideal Mathadhipati. Our Pujya Swamiji’s Αshirvachana of course was a lesson for all of us in humble, but mindful abhivyakti.
Destination two of our yatra was Shri Dungargarh in Rajasthan, a journey covered in 14 long hours on 16th February. This phase of the yatra was exciting for all of us as firstly, this was the first time that all of us had come to Shri Dungargarh. No one knew what to expect. Second, a Shobha Yatra had been planned by the local Saraswat-s. Third, the Char Yama Shivaratri pujana by H.H. Swamiji was going to be held here for the first time.
Next day the Shobha Yatra started at 2.00 p.m. Interestingly, eight of our Αmchi sadhaka-s were given the privilege (a doubtful one, as it turned out!), to ride four horses and four camels. Pujya Swamiji was seated in a chariot drawn by two majestic white horses for the long procession around town. The scorching bright sun did not deter the enthusiasm and hundreds turned out on the streets, joyously celebrating the arrival of their  Guru. After the hour-long Shobha Yatra, a Dharma Sabha organised by the Local Saraswat sadhaka-s commenced with Dipa-Prajwalana and Shri Guru Paduka Pujana, and concluding with the much-awaited Αshirvachana by H.H. Swamiji.
The next day (18th February) was Shiva Ratri with the Char Yama Shiva pujana by Pujya Swamiji scheduled from 10 p.m. to 5.30 a.m. and we considered ourselves fortunate to participate in that. The same morning all our ladies were invited to put on the traditional mehendi on their palms – a traditional custom of the land. The local sadhaka-s - participating in this pujana for the first time, were very excited. They had several queries which were very patiently answered by our senior sadhaka-s.
By evening, the stage was all set. Lord Bhavanishankar was ceremonially brought on to the dais and our Vaidika-s made all necessary preparations. Quite a few local sadhaka-s joined, performing the pujana alongside Pujya Swamiji. Our Parama Pujya Swamiji started the first Yama pujana at 10.00 p.m. The Presence & Grace of H.H. Swamiji performing the pujana, left everyone mesmerised. The atmosphere was very divine and everybody thoroughly enjoyed every moment in spite of the cold night.
After the first Yama pujana, Pujya Swamiji observed Mauna for the next three Yama -s. This gave a very special opportunity to Srikar Baljekar mam, Varun Mallapur mam & Sameer Masurkar mam, who took turns reciting the mantra-s effortlessly, synchronising very well with Pujya Swamiji performing the upachara-s. I was personally very happy and impressed with the tremendous amount of effort that went into this all-night Jagarana and Shiva Αradhana. On completion of the fourth Yama, all of us were blessed with receiving the Tirtha at the Divine Hands of our Beloved Guru.
On the evening of 19th February, a visit to the nearby sand dunes was scheduled along with a visit to a farmhouse. It was pure joy to climb up and slide down the sand dunes! Later, we were all taken to a sprawling farm at Bigga Bass. It was a huge farm cultivating spinach, tomato, cauliflower, radish, mustard, etc. It was unbelievable to see so much greenery spread over such a vast area in Rajasthan. We were witness to the making of typical Bajra Rotla (Bhakri) which were served for dinner, of course with farm-fresh vegetables. All of us thoroughly enjoyed this meal at sunset, consisting of fresh Bajra Rotla, sweet Churma, Gawar bhaji and the oh-so-spicy change to chili-chutney, downing it all with gallons of fresh buttermilk.
Our sadhika-s performed Navaratri Nityapatha and Shri Devi Anushthanam the next morning while we sadhaka-s-did Gayatri Anushthanam, with a few local sadhika-s joining in the upasana. The evening saw us visiting the Dungargarh Market where most of us took in a bit of authentic Rajasthani shopping experience. On return, a talk on “Kailash - Manasarovar Yatra” and “Chitrapur Heritage” was presented by Dr. Chaitanya Gulvady mam. Both talks were very engaging and had the audience enthralled throughout. The local sadhaka-s were very impressed with the sheer number of 96 yatri-s going to Kailash along with H.H. Swamiji. They also had many questions on the weather, distance, etc. which were well-answered by Chaitanya Gulvady mam.
Next evening, an interaction with almost 200 school children was organised for H.H. Swamiji along with Sanskrit Sambhashanam conducted by Shilpa Mudur pachi. Shilpa pachi beautifully enacted the story of a lion cub and his young deer-friend, Imli, with a hand-puppet. It was so heartening to see that the children immediately grasped the story answering her questions quite correctly! Children from a local Gurukul recited quite a few Sanskrit shloka-s. Of course, all children received chocolates from Pujya Swamiji. At the behest of H.H. Swamiji, a prasada bhojana was arranged by the organisers and enjoyed by all children.
Next morning (22nd February), we were taken to see “Gopal Goshala” located at Adsar Bas, Shri Dungargarh which housed a few dozen Gir cows. Pujya Swamiji had quite a few queries which were promptly answered by the owner. We returned in time to attend the concluding Dharma Sabha and were blessed by a wonderful Αshirvachana from H.H. Swamiji.
On 23rd February, H.H. Swamiji and the entourage departed to Haridwar, a distance of approximately 561 kms and 12 hours away by road amidst a tearful farewell by our Saraswat Bandhu Bhagini-s by road. 
Enroute to Haridwar, all of us ate our packed lunch at the temple premises near Bhiwani and reached Haridwar at 9.45 p.m. After a welcome Αmchigele dinner at Vyas Αshram belonging to Kashi Math Samsthana, the yatri-s retired to their allotted rooms.
The main temple at Vyas Αshram is dedicated to Bhagawan Vedavyas. The Vrndavan Samadhi of Pujya Shrimat Sudhindra Thirtha Swamiji of Shri Kashi Math Samsthana is situated next to it. The Αshram is located in a beautiful garden with various colourful flowers in full bloom, a treat to the eyes after the dry lands of Rajasthan. This entire Αshram is situated on the banks of River Ganga with its own Ganga Ghat enabling devotees to take a dip in the sacred Ganga waters at any time of the day. Couple of temples dedicated to Shri Ganga Mata and the Navagraha-s stand guard at the private ghat with an Ashvattha gudi under a huge Ashvattha tree nearby. The Vyasashram has acres of agricultural land where many crops, vegetables and fruit trees are cultivated. We got to see two fully grown rudraksha trees laden with raw rudraksha fruit close to the main temple. The entire Αshram premises with its divine temples amidst greenery, is very scenic, peaceful and serene.
The next morning, after witnessing the Nirmalya visarjana puja and Αrati performed by the current Mathadhipati of Kashi Math, Shrimat Samyamindra Thirtha Swamiji, we visited the Αshram Goshala. The goshala is spread on a 5-acre land, with a cowshed on one side with approximately 40 Jersey cows and a dozen cute calves. Beside the cowshed, a huge farm is devoted to the cultivation of grass for cows as well as for growing vegetables.  It also includes a shed to store dry grass. The milk from this goshala is used at Vyas Αshram for all purposes including pujana. The Kashi Mathadhipati accompanied our Pujya Swamiji and showed Him around. The place was well maintained and clean. In the evening, at an informal interaction, all yatri-s and a few visitors to the Vishwa Saraswat Confluence got to meet H.H. Swamiji.
The next two days, 25th & 26th February, were packed with the programmes organised under the auspices of the Vishwa Saraswat Confluence at Vyas Αshram. This event saw approximately 350-400 visitors from across India. We completed the registration formalities by 10.00 a.m., when both Saraswat Mathadhipati-s - our H.H. Swamiji and the Kashi Math Swamiji arrived for the inaugural session comprising of Veda-Ghosha, Dipa-Prajwalana and Live art demonstrated beautifully by the artist, Shri Vilas Nayak. Post tea-break, our own Shri Nitin Gokarn (Principal Secretary, Govt. of UP) spoke at length on the ‘Glory of Saraswat Heritage - Tradition, Qualities and Culture’. This was a very engaging session with participants asking questions at the end. The next speaker was Major General GD Bakshi who spoke energetically on the history of Saraswati River. Post-lunch, a session was conducted by Professor Amitabh Matoo on ‘Kashmir and Saraswat-s’ to trace the history of migration of Saraswat-s from Kashmir. Next session was by Shri  Siddharth Pai on ‘ The Role of Saraswat Youth in Taking Indian Economy Forward’ which kept the youth at the conference pretty engaged. Later, Shri Dinesh Kamath conducted a session on ‘Sanskrit Being an Essential Part of Our History’. The talk explained how the world’s oldest language had such a profound influence on Indian culture for so many centuries. Post-tea session had our Shri Durgesh Chandavarkar, Chairman of SVC Bank highlighting the value and impact of Saraswat-s in the Banking Sector in providing many opportunities for employment and in the process, creating a huge revenue for so many decades. Next, there was a quiz session related to ‘Know Your History’ conducted by Shri Nandmohan Shenoy. The quiz was quite unusual and a lot of fun since it was a completely digital experience using smartphones. Later, Smt. Smita Shenoy and Smt. Maya Rohit pachi-s presented a joint session on ‘Jewellery and Dresses of North and South India Saraswat-s’. 

After this, all our Αmchi Sadhaka-s and some delegates gathered near the Ganga Ghat for Ganga Pujana by both the Mathadhipati-s. The Vyas Αshram Ghat was brightly lit up for the occasion. The Vaidika-s chanted the mantra-s and both Swamiji -s performed the pujana in sync. It was indeed a once in a lifetime experience seeing Them do the Ganga Αrati at the Ganga Ghat. The onlookers at the event were mesmerised by this rare opportunity, feeling both fortunate and blessed. The evening programme concluded with Αrati at the temples.
The next morning at the Sammelana, the first session was by Smt. Shefali Vaidya on ‘Saraswat Astha and Temples’. It was an interesting presentation depicting and recollecting the history of all the Saraswat temples mostly in and around Goa region. The next session by Shri Sahil Kini, ‘Building a Digital India - A Saraswat Perspective’, talked of his experiences with Shri Nandan Nilekani during the build-up of the Aadhar cards. This gave a very interesting perspective on the projects involving Government officials and private company employees working together and the success they achieved despite coming off different routes. Next session was on ‘Yoga and Happiness’ by Yogacharya Shri Raghvendra Pai who elaborated on Yoga, the ancient Indian way that helps enrich the happiness quotient of people. Later, there was a panel discussion on ‘Saraswat Identity and Unification’ in which our Shri Praveen KadΙe, Shri Uday Gurkar, etc. participated. This was followed by another panel discussion ‘Involvement of Saraswat Youth and Way Forward’. Post-tea, both the Swamiji-s were invited on stage for the concluding Αshirvachana-s. The Sammelana came to an end with melodious Hindustani Classical and devotional music.
On 27th February morning, our yatri-s were taken on a visit to Manasa Devi Temple located high on a mountain and also accessible by a rope-way. A short trip to Rshikesh was also planned where our yatri-s travelled to Shatrughna Ghat for Ganga-darshan and Ram jhula. We did prokshana with the divine Gangajal and chanted Shri Ganga Stotram. Later, we visited the Rshikesh market place, had a sumptuous lunch and got back to Haridwar in the evening to attend the evening Αrati at Vyas Mandir.
On 28th February morning, the yatri-s felt blessed to witness both our H.H. Swamiji and Kashi Math Swamiji performing pujana at the Navagraha Mandir, Ganga Mandir & Ashvatha-Narayana Temple. A few of our sadhaka-s (especially Mahila Varga) got to volunteer for cutting vegetables and fruits and serving lunch at the bhojanashala. In the evening, we were scheduled to see a few temples in Haridwar. We got into the yatri bus by 4.00 p.m. and first visited the Daksheshwar Mahadeva Temple at Kanakhal dedicated to Lord Shiva. This is a beautiful and popular temple (also believed to be a Shakti pitha) which was built by Rani Dhankaur in 1810. Next, we visited the Harihar Αshram belonging to Shri Avadheshanandji Maharaj located at Kanakhal. The main deity at the Αshram is a Parad Shiva-linga (made of Mercury), which weighs approximately 150 kgs. The last stop for the evening was Har ki Pauri (meaning feet of Lord Vishnu), a ghat on the banks of river Ganga which is a very popular destination in Haridwar for the beautiful Samuhika Ganga  Αrati in the evening. Although we could not stay back for the Αrati, the place itself has such peaceful and divine vibes that one can spend hours without ever getting tired. On return, we had an informal interaction with our H.H. Swamiji with a few bhajan-s.
1st March was a truly divine samyoga – our Pujya Guruswami’s Shishya Swikara Divasa co-incided with the Vardhanti of the pratishtha of the Bhagavana Vedavyasa vigraha at the Vedavyasa Temple! In the early morning, Navaratri Nityapatha and Devi Anushthanam as well as the Gayatri Anushthanam were offered respectively by our Mahila and Purusha varga-s. Post-breakfast, we all witnessed the ceremonial divine Abhisheka and Pujana of Bhagavan Vedavyasa performed by both the Mathadhipati-s. The Gandha-lepita Viranjaneya at the Samadhi of H.H. Sudhindra Thirtha Swamiji was a visual treat. This was followed by offering of ‘Patta Kanika’ and receiving Blessings from H.H. Swamiji and the Kashi Math Swamiji. Purnahuti of Laghu Vishnu Havana at Shri Vedavyasa Mandir in the presence of both Swamiji-s was scheduled for 12.30 p.m. We had falahara at the bhojanashala followed by Maha Αrati and lunch. In the evening, there was Dharma Sabha at Vyas Αshram in the divine presence of both Swamiji-s.
On 2nd March morning, all yatri-s attended Pranayama and Ninada sessions taken by Chaitanya Gulvady mam for Purusha varga and Sharayu Haldipur pachi for Mahila varga. All the yatri-s enjoyed these excellent sessions with simple but effective stretching exercises, Kapalabhati and Bhastrika Pranayama followed by Ninada, well-demonstrated and explained. Later, H.H. Swamiji was shown around the vegetable farm at Vyas Αshram temple premises by the Kashi Mathadhisha. There were no programs in the evening and all yatri-s got ready to pack their belongings to resume the yatra next morning.
On 3rd March, all yatri-s after an early breakfast, loaded our luggage in the yatri bus and followed H.H. Swamiji’s convoy on the next long journey to Pushkar 665 kms and 11 hours away.
On our first morning 4th March in Tirtha Sthali Pushkar, we visited the Inderghat at the Pushkar Lake. After a dip in the cool waters of the sacred Pushkar sarovara, Pujya Swamiji performed puja at the banks. Thereafter, we had darshan at the three nearby temples walking through the quaint narrow lanes -The Atmateshwar temple, the Old Rangji temple and the New Rangji temple.
All of us travelled along with H.H. Swamiji the next morning to Bhilwara, about 150 kms away, at the invitation of the local Saraswat-s. H.H. Swamiji was given a rousing welcome by an ecstatic crowd of approximately 250-300 Saraswat devotees. A befitting formal reception followed on stage ending with a beautiful Ashirvachana by H.H. Swamiji in which Pujya Swamiji lucidly explained the Vidyaya, Shraddhaya, Upanishada mantra to the congregation of brahmana-s who take pride in their brahmanatva. The devotees were obviously thrilled with the newly established connect with “Hamare Saraswat Guruji.”
Our third day at Pushkar saw us all by 4 a.m. at the much-awaited morning puja at the precincts of the holy Brahma temple said to be the only Brahma temple in the world. This temple is a 14th century structure with a statue of Chaturmukha Lord Brahma with Devi Gayatri beside Him. Pujya Swamiji arrived at the temple after Snana at Pushkar Sarovara in the brahma muhurta. The Mahant of the Brahma temple received Pujya Swamiji and briefed us on the history of this temple. Our Pujya Swamiji performed arati and all sadhaka-s received prasada with a cup of welcome kulhad vali chai!
The same evening, H.H. Swamiji was felicitated by the local organisers and a few dignitaries of Pushkar. Pujya Swamiji concluded the function with a short but potent Ashirvachana explaining the importance of learning from the Guru, from co-sadhaka-s, then by doing chintana, absorbing the learnings in time. The attentive listeners were very happy with the explanation of the three Devi Anushthanam mantra-s too! At the end, Pujya Swamiji blessed everyone with Tirtha-Vitarana.
Next day (7th March), early morning, Pujya Swamiji and all yatri-s travelled to Jodhpur, 180 kms and approximately 5 hours away. After freshening up, in the evening, H.H. Swamiji was accorded a warm reception at Samvit Dham in the Dharma sabha organised by the Samvit sadhaka-s. In His Ashirvachana, Pujya Swamiji explained the meaning of the bhajana ‘Jayati Girirajeshwari’ – Devi Girirajeshwari being the Adhishthatri deity of Samvit Dham. He emphasised the need for Guru Anusandhana with constant anusmarana in and through vyavahara to receive new inspirations. The sabha concluded with Tirtha-Vitarana by Pujya Swamiji.
On 8th March, all sadhaka-s reached Samvit Dham early in the morning with great excitement as the much-awaited Atirudra yaga was to begin. Quite a few of our Amchi Sadhaka-s had signed up for Shakalya Seva and Sankalpa Seva. The first process was to seek ‘Guru-Ajna’/Anumati. We were witness to this as Pandit Navaratna Agnihotriji informed our H.H. Swamiji and Narayana Giri Swamiji the details regarding conduct of Atirudra and sought their Anumati to start the Yajna. Next was the Shuddhikarana (cleansing) process for all participating Yajamana-s/Sadhaka-s with ten dravya-s e.g. Bhasma, gandha, sand, haldi, kunkuma, Pancha-gavya, etc. Next was the Mandapa Pravesha when all Mahajana-s/Yajamana-s who had registered for Shakalya Seva were requested to enter the Yajna-shala. The Yajna process began with ‘Arani-Manthana’, the very interesting traditional ritual to kindle the agni for the yajna. This was performed in the presence of both H.H. Swamiji and Narayana Giri Swamiji. On this first day, one avartana of the Rudri was completed in the afternoon. H.H. Swamiji conducted Swadhyaya on select shloka-s of ‘Shivanandalahari’ which immediately made our minds more focused and set the tone for participation in the pavitra yajna for the five days.
9th March, the second day of Swadhyaya conducted by our Pujya Swamiji entered deeper levels and attracted many Samvit sadhaka-s to attend. The Swadhyaya had lots of practical guidance for sadhaka-s too, in and through the upadesha picked from the Shivanandalahari shloka-s.
At the yajnashala, the 2nd and 3rd avartana-s were completed. The effect of a dozen Pandita-s reciting the Rudri with great speed and all 41 Havana kunda-s set up for the ahuti-s created an atmosphere that is really hard to describe. One just got engrossed in this entire experience. The evening concluded with a satsanga in the sannidhya of H.H. Swamiji.
On the morning of 10th March, we assembled quickly for the Swadhyaya session continued by H.H. Swamiji. The Atirudra Yajna too continued as planned completing the 4th & 5th avartana-s of Rudripath in the morning. The 6th avartana was chanted in the afternoon. In the evening, Pujya Swamiji performed Devi Pujana in which Narayana Giri Swamiji also participated along with all sadhaka-s. The effect of the Pujana spread a lot of positivity and warmth amongst all sadhaka-s. Later, all Mahatma-s were requested to speak. H.H. Swamiji emphasised that the Anugraha of the Acharya is always there but it will manifest in our lives only when we make a diligent effort in our sadhana.
In the early morning hours of 11th March, a stretching, pranayama & Ninada session was conducted for the Taru-s by Sharayu Haldipur pachi. At the Atirudra, the 7th avartana was completed followed by the 8th avartana in the afternoon. In the evening, a grand Nagarik Abhinandana (Civic Reception) function was organised at Shri Junakhedapati Hanuman temple in Jodhpur to honor our H.H. Swamiji. The Vashishtha Sangeet Mandali offered a swagat geet. The Maharani of Jodhpur, Rani Hemalata Raje felicitated H.H. Swamiji and Narayana Giri Swamiji. Thereafter followed felicitations by other local dignitaries and Heads of Institutions. The atmosphere was electric with representatives from many  institutions present and keen to offer their respect felicitating Pujya Swamiji . The other Mahatma-s attending the function were also felicitated.
After a moving tribute and pravachana by Narayana Giri Swamiji, a Manapatra in Sanskrit was presented to H.H. Swamiji, read out by a Sanskrit professor. The highlight of the evening was the offering to H.H. Swamiji, of a scintillating, precious Sphatika and Rudraksha mala which belonged to Pujya Bade Swamiji, by Narayana Giri Swamiji in humble remembrance of Pujya Bade Swamiji ’s wish. The Prasadika Mala was equally respectfully accepted by H.H. Swamiji in utter humility. The grand function came to an end with Pujya Swamiji’s illuminating Ashirvachana.
12th March, the last day of the Atirudra began with the 9th avartana. The afternoon session was for avartana to other Gods and Goddess and in total, 9 avartana-s of the Rudri were completed. 150 Pandita-s, yajamana-s offered ahuti, preceding the Purnahuti in the evening. Purnahuti was the solemn culmination of the unique and rare Atirudra Yaga. Purnahuti was offered at the Divine Hands of H.H. Swamiji and Narayana Giri Swamiji in the presence of other invited Mahatma-s. This was followed by Mangalarati signifying the successful completion of the Atirudra Yaga sankalpa. The concluding Dharma Sabha began at Samvit Dham with a welcome song by Vashishtha Sangeet Mandali. All the Local Dignitaries and Sponsors who had contributed to the 5 - day event, were given the privilege to felicitate H.H.Swamiji  and  Narayana  Giri Swamiji . On behalf of Shri Chitrapur Math, our Dharmapracharaka Dr.Chaitanya Gulvady mam and Krishnanand HebΙekar mam paid obeisance to Narayana  Giri Swamiji and other Mahatma-s present at the sabha, followed by H.H. Swamiji, Narayana  Giri Swamiji and the other Mahatma-s being honoured by the Adhyaksha of Samvit Dham, Smt.Vasant Usha Raniji. After the much-awaited concluding Ashirvachana-s by H.H. Swamiji and Narayana Giri Swamiji, Pujya Swamiji gave Prasada to all key devotees involved in organising and successfully carrying out the Atirudra yaga. Narayana Giri Swamiji concluded the momentous event with a suggestion to plan for a Koti Rudra Yaga next. Thus, came to an end our very sacred, inspiring and educative sojourn at Samvit Dham, Jodhpur.
Next morning (13th March) saw us all move on to our last destination of this yatra – Sant Sarovar at Mount Abu following H.H. Swamiji and Narayana Giri Swamiji. On the way, we had the good fortune to visit some beautiful monuments at Jodhpur. First on the list was Mandore Gardens, a big garden with historical museums, temples, fountains and a lot of monkeys. It is said that Mandore was the capital of Jodhpur before Mehranagarh fort was built by Rao Jodha. We yatri-s clicked a lot of pictures of temples, inscriptions, paintings and toys built during that era.
Our next stop was Meherangarh Fort, a fort that Rao Jodha built on a hilltop at Jodhpur in 14th century on a 1200 - acre land. We could see this magnificent structure from our bus while still a few miles away. We spent a few hours here along with our hired guide (Ms. Shekhawat) who gave us vivid descriptions and a glowing account of everything about the palace. A few of us also shopped for bangles, books and paintings at their souvenir store.
Finally, we visited the temple of Goddess Chamunda who is the ‘Ishta Devata’ or the Revered Deity of the Jodhpur Royal Family since several centuries. This temple is situated within the palace walls as Rao Jodha had brought the statue over from Mandore Gardens to be installed in the new fort. All of us yatri-s paid obeisance and prayed for the Devi’s blessings. After a sumptuous lunch at Muchawala in Jodhpur, we started for Mount Abu arriving there by night.
Then onwards, from 14th March to 19th March, every morning and evening, we were blessed to witness the Pujana and Arati at the Shri Somnath Mahadev Mandir at Sant Sarovar Ashram. We would then offer the Sadhana Panchakam along with Devi Anushthanam by our Mahila varga and Gayatri Anushthanam by the Purusha varga before lunch. One evening, we all participated in a Vimarsha on “Yatra” conducted by Asha Awasthi pachi. The Vimarsha was very engaging, informative, interactive with experiences shared by all yatri-s as we explored the meaning and reasons for undertaking a yatra, along with many lessons learnt. For me personally, this was a revelation summarising why a yatra was conducted and what was the purpose and objective of a yatra. We were also fortunate to be granted an informal satsanga session with Narayana Giri Swamiji.
On 15th March, we travelled to the Dilwara temple, a group of five Shwetambara Jain temples dating back to the 11 century, built mostly using white marble. All yatri-s appreciated these beautiful and intricately carved structures. In the evening, we all visited Shri Dhuleshwar Mahadev temple, which is dedicated to Lord Shiva. Before leaving the temple, all the yatri-s were treated to a cup of delicious ice-cream by the grace of Narayana Giri Swamiji. On our way back, we found time to go shopping in the vicinity of the beautiful and scenic Nakki Lake which was a great spot for boating, snacks and enjoying the colourful sunset.
The morning of 16th March was reserved for a visit to Gurushikhar and Achaleshwar Mahadev temple. At Gurushikhar Temple, we recited the Gurupaduka stotram and Shri Parijnanashrama Trayodashi. The Achaleshwar Mahadev temple constructed around 9th century is believed to be built around the toe of Lord Shiva and all yatri-s got a divine opportunity to offer jalabhisheka. Our Chaitanya Gulvady mam made a small contribution to the temple on behalf of all yatri-s. Later in the day, we attended the very beautiful Swadhyaya on the Ardhanarishwara Stotram conducted by Pujya Swamiji. The same evening, we visited Girishwar Mahadev temple nearby and performed Ashtamurti Upasana.
On 17th March, all Amchi yatri-s and a few taru Samvit sadhaka-s went on an uphill trek to the Table Top Forest Terrain, a walk of approximately 1.5 kms, to spend some time with nature. The view from the summit of the Table Top hill, was spectacular and we were tempted to sit quietly in meditation for a few minutes. It was a good opportunity for our young yuva-s and taru-s to bond together. At the same time, I personally felt this was an opportunity to reconnect our body and mind with nature. All the sadhaka-s, regardless of age, enjoyed this short trek experience and returned feeling fresh and rejuvenated in time to attend the continuing Swadhyaya by Pujya Swamiji. A visit to the Arbuda Devi Temple was scheduled in the afternoon. This temple has 350 steps to climb up the mountain and a lot of naughty monkeys along the way trying to snatch things from passers-by. Once at the entrance, everybody had to bow down in order to enter the inner sanctum sanctorum. After praying for Arbuda Devi’s blessings, all of us reached Sant Sarovar for the evening Arati. 
The morning of 18th March was very special with a planned visit to Kacholi, the Samadhi Sthala of Pujya Swami Ishwaranand Giriji Maharaj. We drove for approximately 40 kilometres to reach Kacholi village. There we were met by Shri Niranjan Soniji (Kalu bhai), caretaker of the place. All of us sat in front of the divine Samadhi in silent meditation for a few minutes. The Samadhi is beautifully constructed in black Shaligrama stone with eight carvings of Lord Shiva (Ashtamurti) on all four sides and with pure white marble over it. Kalu bhaiji arranged an Arati to be offered on behalf of us yatri-s and helped take a few group photos. Later he explained the significance of this place as to how Bade Swamiji would visit this place whenever he was in Ekanta or during winters. The place has the Aravalli mountains on one side and one could see Dhuleshwar Temple from afar. Kalu bhaiji also informed us that this place was not known to everyone as Bade Swamiji would always like to spend His time here alone and loved this place. Bade Swamiji had selected this place for His Samadhi as He wanted people to visit and appreciate the beauty and serenity of this place. We were also informed how the design and details of the Samadhi including the overall Samadhi structure was carefully planned by Bade Swamiji Himself.

I personally felt that if we were to do some manana on all this, it clearly shows us how all the learned Mahatma-s think and how their vision is so different and their perspective so broad as compared to the rest of the people. Overall, the energy and vibes at this Samadhi was palpably on a different level. As Narayana Giri Swamiji says, “Bade Swamiji is still around and it is for the sadhaka-s to find His Presence still out there and to feel it within.” After a sumptuous breakfast served to us at Kacholi, we returned to Sant Sarovar since the locals of Mt. Abu were scheduled to meet H.H. Swamiji and Narayana Giri Swamiji that morning. A Bhandara for Mahatma-s was also arranged the same afternoon. A few yuva-s and taru-s volunteered to serve food at the Bhandara. Our Mangesh Chickermane mam sponsored the Bhandara and had the good fortune to welcome the Mahatma-s by offering a customary Tilaka and Kanika to all Mahatma-s who had graced the occasion. The Bhandara began with our yuva-s and taru-s doing a great job of serving food, constantly remaining on their toes, attending to everyone. After the Bhandara, it was time for all the other sadhaka-s to partake of the delicious lunch. This was our last day at Sant Sarovar and that night we returned to our rooms with a heavy heart to start packing for our return journey back to Mumbai and Karla the next morning.
On 19th March morning, we loaded our luggage in the bus and headed to Sant Sarovar. All yatri-s, yuva-s and taru-s bid goodbye to each other and our hosts at Sant Sarovar and were busy exchanging numbers to stay in contact going forward. Personally, I had a mixed feeling as on one hand our yatra was in the final stage and we were headed home after 40 days. But on the other hand, all of us had stayed together for so long that we were all bonded practically as one family for the last 40 days with everyone caring, supporting, sharing and helping each other.
Finally, it was time for H.H. Swamiji ’s departure from Sant Sarovar for His return journey to Karla. With a very heavy heart, all of us bid goodbye to our Pujya Swamiji in Whose constant Sannidhya we had spent our last 40 days listening to His Upadesha-s, singing bhajana-s, interacting in informal satsanga-s and having lunch/dinner in His divine Sannidhya at Sant Sarovar.
After Pujya Swamiji ’s convoy left, all of us yatri-s felicitated our two bus drivers, Animay and Manju whom we addressed as our humble heroes. These two youngsters had driven the bus for the last 40 days, across states, spanning different times, different weather, always cooperating and polite, never once complaining. Perhaps the bus too took a cue from their friendly behaviour and never once caused any problem throughout the 6000 km yatra over 40 days across 7 destinations! Our yatri-s contributed a generous tip for both of them as a token of their appreciation, and this was handed over at the sacred Sant Sarovar before this final lap of our unforgettable yatra.
Finally, our bus started on the return journey from Mt. Abu to Karla with a resounding - Om Namah Parvati Pataye Hara Hara Mahadeva!
We left Sant Sarovar at 9.00 a.m. and reached Mumbai at 12.30 a.m., where we dropped a few yatri-s. The rest of us reached Karla at 3.45 a.m.- just 20 minutes after the arrival of H.H.  Swamiji ’s convoy. At 9.30 a.m., our bus left for Shirali with three of our yatri-s, arriving there at 1.00 a.m., thereby completing our Uttar Bharat Yatra – an overwhelming once-in-a-lifetime experience.
On behalf of all of us yatri-s, I offer humble, immensely grateful pranama-s at the Lotus Feet of H.H.  Swamiji for this wonderful opportunity of exposure to varied cultures and experiences, for the many valuable lessons learnt in seva and above all, for the loving Sannidhya of our Beloved Guru every single day of this memorable Yatra!
Om Namah Parvati Pataye Hara Hara Mahadeva!
"""

print(main(text))

['\n', 'Forty', ' ', 'days', ' ', 'in', ' ', 'Divine', ' ', 'Sannidhya', ' ', 'of', ' ', 'the', ' ', 'Guru', ' ', '-', ' ', 'Our', ' ', 'Uttar', ' ', 'Bharat', ' ', 'Yatra', '\n', 'A', ' ', 'Report', ' ', 'by', ' ', 'Vinay', ' ', 'Kalyanpur', '\n\n', 'Preparations', ' ', 'for', ' ', 'the', ' ', 'Uttar', ' ', 'Bharat', ' ', 'Yatra', ' ', 'started', ' ', 'months', ' ', 'in', ' ', 'advance', '!', ' ', 'The', ' ', 'very', ' ', 'thought', ' ', 'was', ' ', 'a', ' ', 'bit', ' ', 'overwhelming', ' ', 'for', ' ', 'me', ' ', '–', ' ', '6000', ' ', 'kms', ' ', 'to', ' ', 'cover', ' ', 'by', ' ', 'road', ' ', 'in', ' ', '40', ' ', 'days', ' ', 'with', ' ', 'seven', ' ', 'unknown', ' ', 'destinations', ' ', 'to', ' ', 'visit', '!', ' ', 'But', ' ', 'I', ' ', 'made', ' ', 'up', ' ', 'my', ' ', 'mind', ' ', 'to', ' ', 'join', ' ', 'only', ' ', 'because', ' ', 'it', ' ', 'was', ' ', 'initiated', ' ', 'by', ' ', 'H', '.', 'H', '.', ' ', 'Swamiji', ' ', 'and', ' ', 'we', ' ', 'would', ' ', 'be', ' ', 't

In [1]:
from diacritics import *
mappings = load_mappings("mappings.txt")

In [2]:
mappings['swamiji']

'swāmījī'

In [3]:
tokens = generate_tokens(text)
translated_tokens = reconstruct_tokens(tokens, mappings)
translated_tokens

NameError: name 'text' is not defined

In [8]:
9
reconstruct_tokens([tokens[9]], mappings)

['Sannidhya']

In [10]:
reconstruct_tokens(["sannidhya"], mappings)

['sānnidhya']